
<center>
    <p style="text-align:center">
    <img alt="arize logo" src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="300"/>
        <br>
        <a href="https://docs.arize.com/arize/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/client_python">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>

# <center>Getting Started with Tracing

This guide demonstrates how to use Arize for monitoring and debugging your LLM using Traces and Spans. You can read more about LLM tracing [here](https://docs.arize.com/arize/llm-large-language-models/llm-traces). In this tutorial, you will use opentelemetry and [openinference](https://github.com/Arize-ai/openinference/tree/main) to instrument our application in order to send traces to Arize.

ℹ️ This notebook requires:
- An OpenAI API key
- An Arize Space & API Key


## Step 1: Install Dependencies 📚
Let's get the notebook setup with dependencies.

In [1]:
# Dependencies needed to instrument your openai application using opentelemetry and openinference
!pip install arize-otel openai openinference-instrumentation-openai opentelemetry-sdk opentelemetry-exporter-otlp 'httpx<0.28'

zsh:1: command not found: pip


## Step 2: Get your API Keys

Copy the Arize API_KEY and SPACE_ID from your Space Settings page (shown below) to the variables in the cell below.

<center><img src="https://storage.googleapis.com/arize-assets/barcelos/Screenshot%202024-11-11%20at%209.28.27%E2%80%AFPM.png" width="700"></center>

In [3]:
import os
from getpass import getpass

if not (SPACE_ID := os.getenv("SPACE_ID")):
    SPACE_ID = getpass("🔑 Enter your Space ID: ")

if not (API_KEY := os.getenv("API_KEY")):
    API_KEY = getpass("🔑 Enter your API Key: ")

Add your OpenAI key below to test OpenAI instrumentation

In [7]:
import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

## Step 3. Add our tracing code
We will be using the arize_otel package to register the URL and authentication parameters to send to Arize using OpenTelemetry. You can see what's under the hood by looking [here](https://docs.arize.com/arize/large-language-models/tracing/auto-instrumentation).

In [5]:
# Import open-telemetry dependencies
from arize_otel import register_otel, Endpoints

# Setup OTEL via our convenience function
register_otel(
    endpoints=Endpoints.ARIZE,
    space_id=SPACE_ID,  # in app space settings page
    api_key=API_KEY,  # in app space settings page
    project_name="tracing-haiku-tutorial",  # name this to whatever you would like
)
# Import the automatic instrumentor from OpenInference
from openinference.instrumentation.openai import OpenAIInstrumentor

# Finish automatic instrumentation
OpenAIInstrumentor().instrument()

Overriding of current TracerProvider is not allowed


## Step 4: Run your LLM application
Let's test our app by asking OpenAI to write a haiku.

In [8]:
import openai

# Run OpenAI completion
openai_client = openai.OpenAI()
response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Write a haiku."}],
    max_tokens=20,
)
print(response.choices[0].message.content)

Autumn leaves falling
Crisp air and golden sunlight
Nature's art on display


Great! Our application works!

## Step 5: Log into Arize and explore your application traces 🚀

Log into your Arize account, and look for the project with the same `project_name`.

<img src="https://storage.googleapis.com/arize-assets/tutorials/images/arize-llm-tracing.png" width="1500" />